In [11]:
from langchain.document_loaders import PyPDFLoader
import json
import re

In [12]:
filename = 'guia-da-ema-obliqua-e-dissimulada.pdf'
loader = PyPDFLoader(filename)
pages = loader.load()

In [13]:
def extract_text(p_begin, p_end, lista_diplomatas):
    texto = ''
    for i in range(p_begin-1, p_end):
        #Junta as páginas e corta os dois primeiros caracteres (número da página analisada)
        texto += pages[i].page_content[2:len(pages[i].page_content)]

    texto = texto.replace('\n', ' ')
    #Em caso de espaços duplos (que são recorrentes no pdf)
    texto = texto.replace('  ', ' ')


    #Selecionando diplomatas que tem redação no excerto analisado
    candidatos = []
    for i in lista_diplomatas:
        if i in texto:
            candidatos.append(i)
    
    
    dict = []
    for i in candidatos:
        if(i == "Anônimo" and len(re.findall(r"Anônimo", texto)) > 1):
            for j in range (len(re.findall(r"Anônimo", texto))):
                dict.append({})        
        else:
            dict.append({})

    
    contcandidatos = len(candidatos)
    x = 0
    for i in range (contcandidatos):
        if(candidatos[x] == "Anônimo" and len(re.findall(r"Anônimo", texto)) > 1):
            contcandidatos += len(re.findall(r"Anônimo", texto)) - 1
            for j in range (len(re.findall(r"Anônimo", texto))):
                dict[i]["Nome"] = "Anônimo"
                i+=1
        else:
            dict[i]["Nome"] = candidatos[x]
        x+=1

    
    dictresp = {}

    #Buscando nota máxima
    maximaind = re.search(r"valor: ", texto).end()
    if(maximaind):
        notamaxima = ''
        while(texto[maximaind] != " "):
            if(texto[maximaind] == ','):
                notamaxima += '.'
            else:
                notamaxima += texto[maximaind]
            maximaind += 1

        dictresp["Nota Máxima"] = float(notamaxima)


    #Buscando nota média
    mediaind = re.search(r"NOTA MÉDIA", texto)
    if(mediaind):
        aux = mediaind.end() + re.search(r"[0-9]", texto[mediaind.end():len(texto)]).start()
        notamedia = ''
        while(texto[aux] != ' '):
            if(texto[aux] == ','):
                notamedia += '.'
            else:
                notamedia += texto[aux]
            aux += 1

        dictresp["Nota Média"] = float(notamedia)
    texto = texto.replace('NOTA MÉDIA ', '')


    #Buscando menor nota (quando existir)
    menorind = re.search(r"MENOR NOTA", texto)
    if(menorind):
        aux = menorind.end() + re.search(r"[0-9]", texto[menorind.end():len(texto)]).start()
        menornota = ''
        while(texto[aux] != ' '):
            if(texto[aux] == ','):
                menornota += '.'
            else:
                menornota += texto[aux]
            aux += 1
        
        dictresp["Menor Nota"] = float(menornota)
    texto = texto.replace('MENOR NOTA ', '')


    #Separando pergunta (a expressão regular usada é particular à edição de 2022, outras edições podem ter formatos diferentes)
    inicio_pergunta = re.search(r"a seguir.", texto).end() + 1
    fim_pergunta = re.search(r"Extensão do texto", texto).start()
    if(inicio_pergunta):
        pergunta = ''
        for i in range(inicio_pergunta, fim_pergunta):
            pergunta += texto[i]

        while (pergunta[-1] == " " or re.search(r"[0-9]", pergunta[-1])):
            pergunta = pergunta[0:-1]

    dictresp["Pergunta"] = pergunta

    contanonimo = 0
    indanonimos = []
    for m in re.finditer(r"Anônimo", texto):
        indanonimos.append(m.start())

    #Aqui atribuímos o os índices de início do candidato e do início da redação que serão importantes para a análise, bem como a nota dos participantes
    for i in range (contcandidatos):
        if(dict[i]["Nome"] == "Anônimo"):
            dict[i]['Inicio_candidato'] = indanonimos[contanonimo]
            aux = indanonimos[contanonimo] + len(dict[i]["Nome"])
            contanonimo += 1
        else:
            dict[i]['Inicio_candidato'] = texto.index(dict[i]['Nome'])
            aux = texto.index(dict[i]['Nome']) + len(dict[i]['Nome'])

        #Extraindo nota
        while(True):
            if(re.match(r"[0-9]", texto[aux])):
                break
            aux += 1
        nota = ''
        while(texto[aux] != ' '):
            if(texto[aux] == ','):
                nota += '.'
            else:
                nota += texto[aux]
            aux += 1

        #Caso especial das questões
        if(re.search(r"linha |linha.", texto[aux:aux+60])):
            aux += re.search(r"linha |linha. ", texto[aux:aux+60]).end()
            aux -= 1

        dict[i]['Inicio_redação'] = aux + 1
        dict[i]['Nota'] = float(nota)


    indices = []
    for i in range (contcandidatos):
        indices.append(int(dict[i]['Inicio_candidato']))
        indices.append(int(dict[i]['Inicio_redação']))
    indices.append(len(texto))
    indices.sort()

    #Extraindo padrão de resposta
    if(re.search(r"PADRÃO DE RESPOSTA", texto)):
        aux = re.search(r"PADRÃO DE RESPOSTA", texto).end() + 1
        padraoresposta = ''

        if(re.search(r"Referências|Referência", texto)):
            for i in range (aux, re.search(r"Referências|Referência", texto).start() - 1):
                padraoresposta += texto[i]
        else:
            for i in range (aux, indices[0]):
                padraoresposta += texto[i]
    dictresp["Padrão de Resposta"] = padraoresposta


    for i in range (1, len(indices), 2):
        redacao = ''
        for j in range(indices[i], indices[i+1]):
            redacao += texto[j]

        while (redacao[-1] == " " or re.search(r"[0-9]", redacao[-1])):
            redacao = redacao[0:-1]
        
        #Atribuindo a redação ao participante
        for j in range (contcandidatos):
            if dict[j]['Inicio_redação'] == indices[i]:
                dict[j]['Redação'] = redacao

    dictresp["Candidatos"] = dict

    return dictresp

In [14]:
nomes = pages[9].page_content.split('\n')

#Separando o os nomes dos participantes brasileiros
diplomatas_brasileiros_aux = []
for i in range (1, len(nomes) - 8):
    if(i != 25): #Retirando string que não é interessante para análise
        diplomatas_brasileiros_aux.append(nomes[i])
#Corrigindo nome "Pedro Guilherme Bastos Menezes de Almeida"
diplomatas_brasileiros_aux[28] = diplomatas_brasileiros_aux[28] + ' ' + diplomatas_brasileiros_aux[29]
diplomatas_brasileiros_aux.pop(29)

#Corrigindo nomes com erros de acentuação e adicionando o candidato anônimo
diplomatas_brasileiros_aux.append("Anônimo")
diplomatas_brasileiros_aux[29] = 'Raíssa Guimarães Carvalho'

#Separando os nomes dos participantes intercambistas
diplomatas_intercambistas = []
for i in range (len(nomes) - 7, len(nomes)):
    diplomatas_intercambistas.append(nomes[i])


diplomatas_brasileiros = []
for i in diplomatas_brasileiros_aux:
    nomeaux = i.split(' ')
    if(len(nomeaux) > 1):
        if(len(nomeaux[1]) > 2):
            nome = " ".join([nomeaux[0], nomeaux[1]])
        else:
            nome = " ".join([nomeaux[0], nomeaux[1], nomeaux[2]])
        diplomatas_brasileiros.append(nome)
    else:
        diplomatas_brasileiros.append(nomeaux[0])

#Corrigindo nome
diplomatas_brasileiros[16] = "Guilherme de Augusto"

In [15]:
#Páginas que ditam o início e o fim do excerto a ser extraído
p_begin = 265
p_end = 273
resp = extract_text(p_begin, p_end, diplomatas_brasileiros)

with open("teste.json", "w", encoding='utf8') as outfile: 
    json.dump(resp, outfile, ensure_ascii=False)